In [ ]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
data = pd.read_csv(r'..\Data\allData.csv')

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
df_race = data[data['sessionName'] == 'Race']

In [ ]:
df_race

In [ ]:
df_race.describe()

In [ ]:
df_race.isna().sum()

In [ ]:
df_race = df_race.drop(['Q1','Q2','Q3'], axis=1)

In [ ]:
df_race['totalTime'].describe()

In [ ]:
df_race['totalTime'] = df_race['totalTime'].fillna(df_race['totalTime'].max())

In [ ]:
df_race = df_race.drop(['TimeInterval','code'], axis=1)

In [ ]:
df_race = df_race.dropna()

In [ ]:
fastest  = []
for i in df_race['fastestLapTime']:
    if i != '192.074':
        a = list(map(float,i.split(':')))
        b = int((a[0]*60 + a[1])*1000)
    else : 
        b = 92074
    fastest.append(b)
df_race['fastestLapTime'] = fastest

In [ ]:
df_race.head()

In [ ]:
df_race.isna().sum()

In [ ]:
df_race.columns

In [ ]:
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor() 

In [ ]:
col = [ 'positionText',
       'status', 'year', 'gpName', 'gpNumber', 'sessionName', 'driverId', 'DriverNationality', 'constructorId', 'constructorNationality']
for elmt in col:
    df_race[col] = df_race[col].astype("category")

In [ ]:
df_features = pd.get_dummies(df_race, columns = ['positionText',
       'status', 'year', 'gpName', 'gpNumber', 'sessionName', 'driverId',
       'DriverNationality', 'constructorId', 'constructorNationality'])

In [ ]:
df_features.columns

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_features, test_size=0.2)
X_train = train.drop(['position'], axis=1)
y_train = train['position']
X_test = test.drop(['position'], axis=1)
y_test = test['position']

In [ ]:
model_rd = random_forest.fit(X_train, y_train)

In [ ]:
model_rd.score(X_test,y_test)

In [ ]:
y_pred = random_forest.predict(X_test) 

In [ ]:
erreurs = abs(y_pred - y_test)
print('Mean Absolute Error:', round(np.mean(erreurs), 2))
mape = 100 * (erreurs / y_test)
print('Mean Absolute Percentage Error :', round(np.mean(mape), 2), '%.')

In [ ]:
importances = model_rd.feature_importances_
indices = np.argsort(importances)
# style du graphique 
plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [X_test.columns[i] for i in indices])
plt.xlabel('Relative Importance')

In [ ]:
# example of training a final regression model
from sklearn.linear_model import LinearRegression


model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

erreurs = abs(y_pred - y_test)
print('Mean Absolute Error:', round(np.mean(erreurs), 2))
mape = 100 * (erreurs / y_test)
print('Mean Absolute Percentage Error :', round(np.mean(mape), 2), '%.')